# Análisis y visualización de datos con python
# 5. Remodelación de datos

    - a) Estructuras de datos 
    - b) Tablas pivote: relacionar variables
    - c) Melt: transformar a tidy data
    - d) Concat: Unir tablas uno a uno
    - e) Merge: Unir tablas muchos a uno

---

## 5.a Estructuras de datos

Un proyecto de datos consta de múltiples pasos, de manera sencilla podemos decir que estos son:
1. Obtención de datos
2. Limpieza de datos
3. Análisis y visualización de datos

Se estima que en un proyecto de datos el 80% del tiempo se dedica a limpiar y preparar los datos. Esto hace que el estructurar los datos de una manera que facilite la limpieza y visualización impacte directamente los resultados del proyecto.

Cómo se menciono anteriormente en la sección de Exploración, los datos ordenados o **tidy data** son una forma estándar de asignar el significado de un conjunto de datos a su estructura. Un conjunto de datos es desordenado u ordenado dependiendo de cómo las filas, columnas y tablas se combinan con las observaciones, variables y tipos. En datos ordenados:
* Cada observación forma una fila.
* Cada variable forma una columna.
* Cada tipo de unidad de observación forma una tabla.

Un ejemplo de datos ordenados es el Histórico de Bitacoras Onomásticas (HBO):

In [1]:
import pandas as pd

filename = 'data_raw/CNB_DOB_BPGS_Respuesta_Solicitud 332163723000249.xlsx'
df = pd.read_excel(filename, #nombre del archivo
                   sheet_name="HBO", #nombre de la hoja
                   index_col='ID' #nombre de la columna del indice
                  )
df['Edad_transcrito'] = pd.to_numeric(df['Edad_transcrito'], errors='coerce') #Limpieza
df

,Numero_progresivo_transcrito,Nombre_completo_transcrito,Primer_apellido,Segundo_apellido,Nombres_propios,Fecha_transcrito,Fecha_estandar,Expediente_SEMEFO_transcrito,Procedencia_transcrito,Procedencia_estandar,...,Diagnostico_estandar,Diagnostico_extendido,Sexo,Edad_transcrito,Tipo_restos,Bitacora_ingresos,Pagina_PDF,Foja_transcrito,Observaciones,Conocido_desconocido
ID,,,,,,,,,,,,,,,,,,,,,
BO_1968_00001,S-D,acosta ortega teresa,acosta,ortega,teresa,1968-01-03 00:00:00,1968-01-03,37,S-D,S-D,...,S-D,sin datos,Femenino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00002,S-D,avila de cuestas catalina,avila,de cuestas,catalina,1968-01-05 00:00:00,1968-01-05,58,S-D,S-D,...,S-D,sin datos,Femenino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00003,S-D,arzate paredes juan,arzate,paredes,juan,1968-01-07 00:00:00,1968-01-07,83,S-D,S-D,...,S-D,sin datos,Masculino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00004,S-D,alvarez martinez isaac,alvarez,martinez,isaac,1968-01-07 00:00:00,1968-01-07,86,S-D,S-D,...,S-D,sin datos,Masculino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
BO_1968_00005,S-D,arellano viuda de campos ma.,arellano,viuda de campos,ma.,1968-01-07 00:00:00,1968-01-07,88,S-D,S-D,...,S-D,sin datos,Femenino,NaN,Cadáver,semefo_df_bo_1968,2,1,NaN,conocido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BO_1982_07489,S-D,placenta,s-d,s-d,s-d,1982-06-05 00:00:00,1982-06-05,3079,15a,15A,...,S-D,sin datos,S-D,NaN,Miembros,semefo_df_bo_1982,251,156,NaN,desconocido
BO_1982_07490,S-D,5 dedos del pie derecho de desconocido,s-d,s-d,s-d,1982-06-05 00:00:00,1982-06-05,3060,32a,32A,...,S-D,sin datos,S-D,NaN,Miembros,semefo_df_bo_1982,251,156,NaN,desconocido
BO_1982_07491,S-D,dedo de desconocido,s-d,s-d,s-d,1982-11-19 00:00:00,1982-11-19,6389,32a,32A,...,S-D,sin datos,S-D,NaN,Miembros,semefo_df_bo_1982,251,156,NaN,desconocido


Al trabajar con conjuntos de datos ordenados es importante definir exactamente que es la observación. Por ejemplo, en el HBO se puede suponer que cada observación o fila corresponde a una persona. Sin embargo, esto no es correcto, ya que cada observación corresponde a un ingreso registrado en el SEMEFO, lo cual incluye cadaveres, miembros y fetos. Por lo tanto, en teoría puede haber el ingreso del miembro de una persona en una fecha y el posterior ingreso del cadaver de la misma persona en otra fecha, lo cual corresponderia a dos observaciones aunque sea una misma persona.

Otro caso donde es importante definir la observación es en los estudios longitudinales, donde se estudia al mismo grupo de objetos a lo largo del tiempo. Por ejemplo, si estamos estudiando el efecto de una droga en ratas a lo largo de un año y se toman mediciones todas las semanas, las observaciones no son las ratas en si, sino las mediciones semanales, las cuales incluyen el individuo y la fecha cómo variables.

## 5.b Tablas pivote: relacionar variables

Las tablas pivote o tabla dinámica permiten reorganizar y resumir datos de forma que faciliten su análisis al relacionar dos o mas variables. Son útiles para calcular estadísticas resumidas como sumas, medias o conteos agrupados por diferentes categorías. Esto se puede hacer en muchas hojas de cálculo, en `pandas` usaremos la función `pd.pivot_table()`.

La estructura típica de una tabla pivote se compone de las siguientes partes:

* Índice (Filas): Las filas representan las categorías principales por las que se agrupa el conjunto de datos. En una tabla pivote de ventas, por ejemplo, las filas podrían ser los nombres de los productos o las regiones de venta.
* Columnas: Las columnas representan las variables o categorías adicionales por las cuales se distribuyen los datos. Siguiendo el ejemplo de ventas, las columnas podrían ser trimestres o meses.
* Valores: Los valores son los datos que se resumen en la intersección entre filas y columnas. Estos valores pueden ser sumas, promedios, conteos o cualquier otra función de agregación aplicada a los datos originales.
* Campos adicionales (Opcionales): Algunas tablas pivote incluyen filtros adicionales para permitir una segmentación dinámica de los datos, como seleccionar solo un rango de fechas o una categoría específica.

Por ejemplo, supongamos que se quiere ver la relación entre el tipo de restos y el sexo. Lo primero que se debé de hacer es imaginar cómo se vera la tabla, colocando las filas, columnas y valores.

|         | Mujer | Hombre | S-D |
|---------|------|------|------|
| Cadáver | ###  | ###  | ###  |
| Miembros| ###  | ###  | ###  |
| Feto    | ###  | ###  | ###  |

donde '###' es el número de ingresos de ese tipo de restos en el año

A continuación es necesario determinar donde esta la información en el conjunto de datos ordenado o si no existe calcularla.

* Filas: Tipo_restos
* Columnas: Sexo
* Función: función `size()`

En base a eso podemos escribir la función correspondiente.

**Nota**: La función  `count` es muy similar a `size` pero se aplica a cada columna y no cuenta los `nan` , por lo que los valores pueden variar.

In [2]:
# tabla pivote
pt_restos_sexo = pd.pivot_table(df, index='Tipo_restos', columns='Sexo', aggfunc='size')
pt_restos_sexo

Sexo,Femenino,Masculino,S-D
Tipo_restos,,,
Cadáver,19503,71182,569
Feto,462,662,1720
Miembros,384,1511,170
Recién nacido,251,379,3
Restos óseos,1,7,40


Es posible generar tablas pivote mucho más complicadas a partir de un conjunto de datos ordenados. Por ejemplo, supongamos que nos interesa saber si hay diferencias en la edad promedio los de ingresos conocidos y desconocidos de cada sexo a lo largo de los años.

La tabla se vería:

|                    |  1973  |  1974  |  1975  |
|--------------------|--------|--------|--------|
| Mujer conocida     | p_edad | p_edad | p_edad |
| Mujer desconocida  | p_edad | p_edad | p_edad |
| Hombre conocido    | p_edad | p_edad | p_edad |
| Hombre desconocido | p_edad | p_edad | p_edad |

donde 'p_edad' es el promedio de edad

A continuación es necesario determinar donde esta la información en el conjunto de datos ordenado. Si no existe directamente se puede calcular

* Filas: Sexo y Conocido_desconocido
* Columnas: Año obtenido de Fecha_estandar
* Values: Edad_trasncrito
* Función: `mean`

Se pueden usar varios variables o columnas para generar la tabla pivote. Esto genera un [multindice](https://pandas.pydata.org/docs/user_guide/advanced.html#). 

In [3]:
# Calculemos el año
df['Año'] = df['Fecha_estandar'].dt.year 
# Tabla pivote
pt_sexocon_año_edadmean = pd.pivot_table(df, 
                                index=['Sexo','Conocido_desconocido'], 
                                columns='Año', 
                                values='Edad_transcrito',
                                aggfunc='mean')
pt_sexocon_año_edadmean

Año                             1973.0     1974.0     1975.0     1976.0  \
Sexo      Conocido_desconocido                                            
Femenino  conocido                 NaN  39.101967  38.101181  39.472951   
          desconocido              NaN  43.990196  18.048649  26.474752   
Masculino conocido                53.0  36.293636  36.693664  36.977398   
          desconocido              NaN  42.531464  36.187006  36.161958   
S-D       conocido                 NaN        NaN        NaN        NaN   
          desconocido              NaN        NaN  10.400000   4.166667   

Año                                1977.0     1978.0     1979.0  1980.0  
Sexo      Conocido_desconocido                                           
Femenino  conocido              40.115475  47.376248  38.598319     NaN  
          desconocido           21.061224  19.808081  25.651934     NaN  
Masculino conocido              36.341400  41.630306  36.675401     NaN  
          desconocido           36.331971  37.511224  38.538830    35.0  
S-D       conocido               8.000000        NaN        NaN     NaN  
          desconocido           11.600000   4.500000  18.125000     NaN

Podemos volver el multindice en columnas con la función `.reset_index()`.

In [4]:
pt_sexocon_año_edadmean.reset_index( inplace=True)
pt_sexocon_año_edadmean

Año,Sexo,Conocido_desconocido,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0
0,Femenino,conocido,NaN,39.101967,38.101181,39.472951,40.115475,47.376248,38.598319,NaN
1,Femenino,desconocido,NaN,43.990196,18.048649,26.474752,21.061224,19.808081,25.651934,NaN
2,Masculino,conocido,53.0,36.293636,36.693664,36.977398,36.341400,41.630306,36.675401,NaN
3,Masculino,desconocido,NaN,42.531464,36.187006,36.161958,36.331971,37.511224,38.538830,35.0
4,S-D,conocido,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,NaN
5,S-D,desconocido,NaN,NaN,10.400000,4.166667,11.600000,4.500000,18.125000,NaN


Esta no es la única forma de obtener tablas derivadas, más adelante en el curso trataremos el tema de group_by (split-apply-combine) más a detalle.

Existen una gran cantidad de funciones que se pueden usar. Es recomendable revisar la documentación de [funciones de agrupamiento](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#seriesgroupby-computations-descriptive-stats).
Algunas funciones útiles son: 
* mean: promedio
* median: mediana
* std: desviación estandar
* sum: suma de valores
* size: tamaño del grupo
* count: conteo del grupo
* first: primer valor
* last: último valor
* nth: n-esimo valor
* min: valor mímino
* max: valor máximo

Además se pueden usar funciones de otras bibliotecas como `numpy` o definir funciones especiales.
Por ejemplo, la siguiente función regresa los strings más comunes de una columna.

In [5]:
def textos_mas_comunes(series, n=5, sep=', '):
    textos = series.value_counts().head()
    textos = textos.index
    textos = sep.join(textos)
    return textos

pt_sexocon_restos_nombrecomun = pd.pivot_table(df, 
                                   index=['Sexo','Conocido_desconocido'], 
                                   columns='Tipo_restos', 
                                   values='Nombres_propios',
                                   aggfunc=textos_mas_comunes)
pt_sexocon_restos_nombrecomun

Tipo_restos                                                               Cadáver  \
Sexo      Conocido_desconocido                                                      
Femenino  conocido                     maria, guadalupe, juana, margarita, carmen   
          desconocido                        s-d, juana, maria, guadalupe, ofelia   
Masculino conocido                          jose, juan, francisco, antonio, jesus   
          desconocido                        s-d, francisco, juan, antonio, pedro   
S-D       S-D                                                                 s-d   
          conocido                          s-d, guadalupe, cruz, "n", concepcion   
          desconocido           s-d, guadalupe, loreto, feto (r.n.) 738, el al...   

Tipo_restos                    Feto       Miembros Recién nacido Restos óseos  
Sexo      Conocido_desconocido                                                 
Femenino  conocido              s-d            s-d           s-d          NaN  
          desconocido           s-d      s-d, gina           s-d          s-d  
Masculino conocido              s-d   s-d, gerardo           s-d          NaN  
          desconocido           s-d  s-d, salvador           s-d          s-d  
S-D       S-D                   NaN            NaN           NaN          NaN  
          conocido              s-d            s-d           NaN          NaN  
          desconocido           s-d            s-d           s-d          s-d

### Ejercicio 1

¿Cómo varia la edad mínima, mediana y máxima por tipo de restos y sexo?

NOTA: Para responder esta pregunta es necesario usar varias tablas pivote.

## 5.c Melt: transformar a tidy data

Una de las herramientas más usadas para el análisis de datos es Excel, ya que es muy potente y facíl de usar. Sin embargo, muchas veces al diseñar las tablas esto se hace mezlcando la obtención, limpieza y análisis de datos en un solo paso, lo cual puede dificultar su posterior manejo.

Es común que las tablas de Excel no estén en formato tidy data, sino que estén en alguna variación similar a una tabla pivote. En ese caso es necesario reestructurar la tabla a formato tidy data.

Es importante destacar que existen muchos formatos de datos desordenados, por lo que puede ser necesario realizar una serie de correciones manuales y automáticas dependiendo de la situación. El artículo original de [tidy data](http://vita.had.co.nz/papers/tidy-data.pdf) cubre varios ejemplos comunes. 
AIs generativas como ChatGPT y DeepSeek también pueden servir para obtener el código para reestructurar los datos, pero es necesario ser cuidadoso al escribir el prompt y revisar a detalle los resultados.

La función `melt` en `pandas` es útil para convertir datos en formato ancho (wide) como el de uan tabla pivote a formato largo (long) cómo el de tidy data. Cuando usamos `melt`, seleccionamos una o más columnas como identificadores y convertimos las demás columnas en dos columnas nuevas: una para los nombres de las variables (`variable`) y otra para sus valores (`value`).

La función `melt` tiene los siguientes parámetros clave:
- `id_vars`: Columnas que actúan como identificadores y no deben transformarse.
- `value_vars`: Columnas que se convertirán en las nuevas columnas de `variable` y `value`. Si no se especifica, se utilizan todas las columnas que no están en `id_vars`.
- `var_name`: Nombre personalizado para la columna de variables.
- `value_name`: Nombre personalizado para la columna de valores.

Por ejemplo, supongamos que se quiere transformar la tabla pivote `pt_sexocon_año_edadmean` a una tabla en formato tidy data.

| Sexo  | Cono_descono |  1973  |  1974  |  1975  |
|-------|--------------|--------|--------|--------|
| Mujer | conocida     | p_edad | p_edad | p_edad |
| Mujer | desconocida  | p_edad | p_edad | p_edad |
| Hombre| conocido     | p_edad | p_edad | p_edad |
| Hombre| desconocido  | p_edad | p_edad | p_edad |

El primer paso es definir cuales son las variables y cuales son las observaciones.

* Variables: Sexo, Conocido_desconocido, Año, p_edad
* Observación: cada año se midio la edad promedio para cada grupo de personas

A continuación se determina que columnas es necesario reestructurar y cuales no,

* NO transformar (`id_vars`): Sexo, Conocido_desconocido
* Transformar (`value_vars`): años (1974, etc), es decir las columnas restantes

Tambien es buen momento de decidir un nombre para la columna del valor.

En base a eso podemos escribir la función correspondiente

In [6]:
df_edadmean = pt_sexocon_año_edadmean.melt(
                                      id_vars=['Sexo', 'Conocido_desconocido'],
                                      value_name='Edad_promedio'
                                      )
df_edadmean

,Sexo,Conocido_desconocido,Año,Edad_promedio
0,Femenino,conocido,1973.0,NaN
1,Femenino,desconocido,1973.0,NaN
2,Masculino,conocido,1973.0,53.000000
3,Masculino,desconocido,1973.0,NaN
4,S-D,conocido,1973.0,NaN
5,S-D,desconocido,1973.0,NaN
6,Femenino,conocido,1974.0,39.101967
7,Femenino,desconocido,1974.0,43.990196
8,Masculino,conocido,1974.0,36.293636
9,Masculino,desconocido,1974.0,42.531464


## 5.d Concat: Unir tablas uno a uno

Es posible unir tablas de distintas maneras usando pandas, en este tutorial veremos solo las más sencillas, pero les recomendamos ver la guia de [Merge, join, concatenate and compare](https://pandas.pydata.org/docs/user_guide/merging.html).

Por ejemplo, estas series muestran la cantidad de ingresos por tipo de restos y sexo para menores y mayores de edad.

In [7]:
df_restossexo_minedad = df.loc[df['Edad_transcrito']<18, ['Tipo_restos','Sexo']] \
                          .value_counts()#.reset_index()
df_restossexo_minedad.name = 'N_menores_edad'
df_restossexo_minedad

Tipo_restos    Sexo     
Cadáver        Masculino    3109
               Femenino     1493
Feto           Masculino     409
               Femenino      303
Recién nacido  Femenino       80
               Masculino      80
Feto           S-D            44
Miembros       Masculino      29
               Femenino       17
Cadáver        S-D             1
Restos óseos   Masculino       1
Name: N_menores_edad, dtype: int64

In [8]:
df_restossexo_maxedad = df.loc[df['Edad_transcrito']>=18, ['Tipo_restos','Sexo']] \
                          .value_counts()#.reset_index()
df_restossexo_maxedad.name = 'N_mayores_edad'
df_restossexo_maxedad

Tipo_restos    Sexo     
Cadáver        Masculino    25348
               Femenino      6078
Miembros       Masculino      224
               Femenino        54
Feto           Masculino        6
Cadáver        S-D              5
Feto           Femenino         4
               S-D              1
Recién nacido  Femenino         1
Restos óseos   Masculino        1
Name: N_mayores_edad, dtype: int64

Una forma de unir dos tablas es con la función `concat()`. Esta función toma una lista de dataframes y las une por el índice o las columnas. En este caso es una unión uno a uno, es decir, cada valor del indice o columna es único y no se repiten.
Es importante notar que, aunque los valores no se repiten, los indices si se repiten, por ejemplo ('Feto', 'Masculino') aparece dos veces.
Esta operación puede ser vista como apilar las tablas a lo alto (filas), uniendo a lo ancho (columnas).

In [9]:
pd.concat([df_restossexo_minedad,df_restossexo_maxedad])

Tipo_restos    Sexo     
Cadáver        Masculino     3109
               Femenino      1493
Feto           Masculino      409
               Femenino       303
Recién nacido  Femenino        80
               Masculino       80
Feto           S-D             44
Miembros       Masculino       29
               Femenino        17
Cadáver        S-D              1
Restos óseos   Masculino        1
Cadáver        Masculino    25348
               Femenino      6078
Miembros       Masculino      224
               Femenino        54
Feto           Masculino        6
Cadáver        S-D              5
Feto           Femenino         4
               S-D              1
Recién nacido  Femenino         1
Restos óseos   Masculino        1
dtype: int64

En segundo lugar uniremos usando `pd.concat(axis=1)`, es decir, los valores se uniran en las filas.
Nota como los nombres de las columnas se mantuvieron como estaban en las tablas originales.
Esta operación puede ser vista como pegar las tablas a lo ancho (columnas), uniendo a lo alto (filas).

In [10]:
pd.concat([df_restossexo_minedad,df_restossexo_maxedad], axis=1)

N_menores_edad  N_mayores_edad
Tipo_restos   Sexo                                     
Cadáver       Femenino             1493          6078.0
              Masculino            3109         25348.0
              S-D                     1             5.0
Feto          Femenino              303             4.0
              Masculino             409             6.0
              S-D                    44             1.0
Miembros      Femenino               17            54.0
              Masculino              29           224.0
Recién nacido Femenino               80             1.0
              Masculino              80             NaN
Restos óseos  Masculino               1             1.0

### Ejercicio 2

Transforma el dataframe `pt_sexocon_restos_nombrecomun` a formato tidy data.

## 5.e Merge: Unir tablas muchos a uno

Otro caso de unir tablas es cuando tenemos muchos valores que se uniran a uno solo.

Por ejemplo, supongamos que tenemos la siguiente tabla con información de los diagnósticos más comunes. Esta tabla nos da información adicional que nos gustaría unir a la informacíon contenida en el HBO. 

In [11]:
data_diag = [['S-D','sin datos','SD'],
             ['TM','traumatismo multiple','Trauma'],
             ['TCE','traumatismo craneo encefalico','Trauma'],
             ['BN','bronconeumonia','Neumonías'],
             ['CVG','congestion visceral generalizada','Falla de órganos'],
             ['HPAFC','herida arma fuego craneo','Herida por arma de fuego'],
             ['Dispensa','dispensa','SD'],
             ['AOVA','asfixia por obstruccion de vias aereas','Obstrucción de vías aéreas'],
             ['CH','cirrosis hepatica','Cirrosis'], ]
data_diag = pd.DataFrame(data_diag, columns=['Diagnostico','Extendido','Clasificación'])
data_diag

,Diagnostico,Extendido,Clasificación
0,S-D,sin datos,SD
1,TM,traumatismo multiple,Trauma
2,TCE,traumatismo craneo encefalico,Trauma
3,BN,bronconeumonia,Neumonías
4,CVG,congestion visceral generalizada,Falla de órganos
5,HPAFC,herida arma fuego craneo,Herida por arma de fuego
6,Dispensa,dispensa,SD
7,AOVA,asfixia por obstruccion de vias aereas,Obstrucción de vías aéreas
8,CH,cirrosis hepatica,Cirrosis


Por otro lado, podemos calcular los diagnosticos más comunes:

In [12]:
df_diag = df['Diagnostico_transcrito'].value_counts().head(10).reset_index()
df_diag.columns = ['Diagnostico','Freq']
df_diag

,Diagnostico,Freq
0,S-D,55878
1,TM,7505
2,TCE,5939
3,BN,1826
4,CVG,1287
5,TCT,1176
6,HPAFPC,833
7,Dispensa,827
8,AOVA,791
9,BN+CH,715


Ahora, ambas tablas comparten la columna 'Diagnostico', por lo que es posible unirlas en una sola.

Existen varias formas de unir tablas:
* **`inner` (por defecto)**: Devuelve las filas que coinciden en ambas tablas.
* **`outer`**: Devuelve todas las filas, rellenando con valores `NaN` donde no hay coincidencias.
* **`left`**: Devuelve todas las filas de la tabla izquierda y las coincidencias de la tabla derecha.
* **`right`**: Devuelve todas las filas de la tabla derecha y las coincidencias de la tabla izquierda.

En este caso usaremos la opción `how='outer'`. Nota como la tabla resultante mide 12 x 4. Las cuatro columnas son la unión de la tabla de frecuencia de diagnósticos más la tabla con los diagnosticos extendidos. Por otro lado, las 12 columnas son el resultado de que algunos diagnosticos aparecen solo en la tabla de frecuencias (TCT, HPAFPC, BN+CH) y otros solo aparecen en la tabla extendida (HPAFC, CH). En los casos donde no hay 

In [13]:
df_diag.merge(data_diag, how='outer', on='Diagnostico')

,Diagnostico,Freq,Extendido,Clasificación
0,S-D,55878.0,sin datos,SD
1,TM,7505.0,traumatismo multiple,Trauma
2,TCE,5939.0,traumatismo craneo encefalico,Trauma
3,BN,1826.0,bronconeumonia,Neumonías
4,CVG,1287.0,congestion visceral generalizada,Falla de órganos
5,TCT,1176.0,NaN,NaN
6,HPAFPC,833.0,NaN,NaN
7,Dispensa,827.0,dispensa,SD
8,AOVA,791.0,asfixia por obstruccion de vias aereas,Obstrucción de vías aéreas
9,BN+CH,715.0,NaN,NaN


Existen muchas formas de unir tablas usando llaves, una vez mas recomendamos ver el tutorial de [merge, join, concatenate and compare](https://pandas.pydata.org/docs/user_guide/merging.html).

### Ejercicio 3:
Une `df_diag` y `data_diag` con la función `merge()` usando los parametros: `inner`, `left` y `right`. Para cada caso detalla el tamaño de la tabla resultante y que diagnosticos de que tabla faltan.

**Gracias**